In [10]:
import open3d as o3d
import numpy as np
import config
import random

In [4]:
pcl = o3d.io.read_point_cloud("pointcloud.ply")
o3d.visualization.draw_geometries([pcl])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


Remove the ground plane

In [9]:
points = o3d.utility.Vector3dVector(np.array([
	[config.point_cloud_range["x_min"], config.point_cloud_range["y_min"], config.min_box_height],
	[config.point_cloud_range["x_max"], config.point_cloud_range["y_max"], config.point_cloud_range["z_max"]]
]))
box_pcl = pcl.crop(
	bounding_box = o3d.geometry.AxisAlignedBoundingBox.create_from_points(points)
)
o3d.visualization.draw_geometries([box_pcl])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [15]:
def fit_plane_vec_constraint(norm_vec, pts, thresh=0.05, n_iterations=300):
	best_eq = []
	best_inliers = []

	n_points = pts.shape[0]
	for iter in range(n_iterations):
		id_sample = random.sample(range(0, n_points), 1)
		point = pts[id_sample]
		d = -np.sum(np.multiply(norm_vec, point))
		plane_eq = [*norm_vec, d]
		pt_id_inliers = get_plane_inliers(plane_eq, pts, thresh)
		if len(pt_id_inliers) > len(best_inliers):
			best_eq = plane_eq
			best_inliers = pt_id_inliers

	return best_eq, best_inliers

def get_plane_inliers(plane_eq, pts, thresh=0.05):
	pt_id_inliers = []
	dist_pt = get_pts_distances_plane(plane_eq, pts)

	# Select indexes where distance is bigger than the threshold
	pt_id_inliers = np.where(np.abs(dist_pt) <= thresh)[0]
	return pt_id_inliers

def get_pts_distances_plane(plane_eq, pts):
	dist_pt = (plane_eq[0] * pts[:, 0] + plane_eq[1] * pts[:, 1] 
			+ plane_eq[2] * pts[:, 2] + plane_eq[3])\
			/ np.sqrt(plane_eq[0] ** 2 + plane_eq[1] ** 2 + plane_eq[2] ** 2)
	return dist_pt

Remove the top plane

In [17]:
points_np = np.asarray(box_pcl.points)
top_plane_eq, top_plane_inliers = fit_plane_vec_constraint([0, 0, 1], points_np, 0.01, 30)

top_plane = box_pcl.select_by_index(top_plane_inliers)
top_side_pcl = top_plane
height = abs(top_plane_eq[3])

points = o3d.utility.Vector3dVector(np.array([
	[config.point_cloud_range["x_min"], config.point_cloud_range["y_min"], config.point_cloud_range["z_min"]],
	[config.point_cloud_range["x_max"], config.point_cloud_range["y_max"], height - config.min_box_height]
]))
box_sides_pcl = box_pcl.crop(
	bounding_box = o3d.geometry.AxisAlignedBoundingBox.create_from_points(points)
)

o3d.visualization.draw_geometries([box_sides_pcl])

In [ ]:
def fit_vertical_plane(norm_vec, pts, thresh=0.05, n_iterations=300):
	best_eq = []
	best_inliers = []

	n_points = pts.shape[0]
	for iter in range(n_iterations):
		id_sample = random.sample(range(0, n_points), 1)
		point = pts[id_sample]
		d = -np.sum(np.multiply(norm_vec, point))
		plane_eq = [*norm_vec, d]
		pt_id_inliers = get_plane_inliers(plane_eq, pts, thresh)
		if len(pt_id_inliers) > len(best_inliers):
			best_eq = plane_eq
			best_inliers = pt_id_inliers

	return best_eq, best_inliers

In [22]:
plane_eq, plane_inliers = box_sides_pcl.segment_plane(distance_threshold=0.01, ransac_n=3, num_iterations=100)
side_pcl = box_sides_pcl.select_by_index(plane_inliers)
side_pcl.paint_uniform_color([1, 0, 0])

o3d.visualization.draw_geometries([side_pcl, box_sides_pcl])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
